In [ ]:
import numpy as np
from dynamicslib.common import *
from dynamicslib.consts import *
from dynamicslib.continuation import *
from dynamicslib.targeter import *
from dynamicslib.plotting import *

In [ ]:
def xtf2X(x0, tf):
    return np.array([x0[0], x0[-2], tf / 2])


def X2xtf(X):
    return np.array([X[0], 0, 0, 0, X[1], 0]), X[-1] * 2


def stmeom2DF(eomf, stm):
    dF = np.array(
        [[stm[1, 0], stm[1, -2], eomf[1]],
         [stm[-3, 0], stm[-3, -2], eomf[-3]]]
    )
    return dF


def f_func(x0, tf, xf):
    return np.array([xf[1], xf[-3]])


func = lambda X: f_df_CR3_single(X, X2xtf, stmeom2DF, f_func, False, muEM, 1e-12)

In [ ]:
# get the linear IC
def get_IC(Lpt, mu=muEM):
    assert Lpt in [1, 2, 3]
    xL = get_Lpts(mu)[0, Lpt - 1]
    step = 1e-3
    eq_soln = np.array([xL, 0, 0, 0, 0, 0])
    A = get_A(eq_soln, mu)
    evals = np.linalg.eigvals(A)
    eigmag = np.imag(evals[np.abs(np.real(evals)) < 1e-10][0])
    beta3 = (eigmag**2 + A[3, 0]) / (2 * eigmag)
    vy0 = -beta3 * eigmag * step

    period = 2 * np.pi / eigmag

    x0 = np.array([xL + step, 0, 0, 0, vy0, 0])
    X0 = xtf2X(x0, period)
    return X0

In [ ]:
Xs1 = []
e1 = []
Xs1, e1 = arclen_cont(get_IC(1), func, [1, 0, 0], 5e-3, 4.2, 1e-11, modified=True)

In [ ]:
Xs2 = []
e2 = []
Xs2, e2 = arclen_cont(get_IC(2), func, [-1, 0, 0], 1e-2, 10, 1e-10, modified=True)

In [ ]:
Xs2b, e2b = arclen_cont(Xs2[-1], func, Xs2[-1] - Xs2[-2], 2e-2, 5, 1e-10, modified=True)

In [ ]:
Xs2c, e2c = arclen_cont(
    Xs2b[-1], func, Xs2b[-1] - Xs2b[-2], 3e-2, 10, 1e-10, modified=True
)

In [ ]:
Xs2d, e2d = arclen_cont(
    Xs2c[-1], func, Xs2c[-1] - Xs2c[-2], 5e-2, 15, 1e-10, modified=True
)

In [ ]:
Xs3 = []
e3 = []
X0 = get_IC(3)
Xs3, e3 = arclen_cont(X0, func, np.array([1, 0, 0]), 3e-2, 17, 1e-8, modified=True)

In [ ]:
import pandas as pd
df = pd.DataFrame(data[0], columns=param_names).set_index("Index")
df.to_csv("L1 Lyapunov.csv")

In [ ]:
es = [e1, e2, e3]
XsAll = [Xs1[::], [*Xs2, *Xs2b, *Xs2c, *Xs2d], Xs3[::]]

param_names = [
    "Index",
    "Initial x",
    "Initial vy",
    "Jacobi Constant",
    "Period",
    "Stability Index",
    "Eig1",
    "Eig2",
    "Eig3",
    "Eig4",
]

xyzs = []
data = []


for Lp in range(3):
    xyzs_lp = []
    data_lp = []
    iterator = list(enumerate(zip(XsAll[Lp], es[Lp])))
    for i, (X, evals) in tqdm(iterator):
        Xcp = X.copy()
        xyzs_lp.append(prop_ic(Xcp, X2xtf))
        args = np.argsort(np.abs(evals))
        eigs = evals[[*args[:2], *args[-2:]]]
        stabind = np.max([np.abs(lam) + 1 / np.abs(lam) / 2 for lam in evals])
        data_lp.append([i, *Xcp[:-1], *get_JC_tf(Xcp, X2xtf), stabind, *eigs])
    xyzs.append(xyzs_lp)
    data.append(data_lp)